In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Plant Pathology 2020 — FGVC7
**Proje:** Elma yaprağı hastalığı sınıflandırma (healthy, multiple_diseases, rust, scab)  
**Yazar:** Ferhat AĞIN 
**Tarih:** 04.06.2025

**Özet:** Bu notebook Kaggle üzerinde Plant Pathology 2020 veri seti kullanılarak bir CNN projesi yapmaya çalıştım.
- Veri yükleme ve EDA
- Veri ön işleme ve Augmentation
- Baseline CNN + Transfer Learning (ResNet50)
- Eğitim, değerlendirme (accuracy/loss, confusion matrix, classification report)
- Grad-CAM ile açıklanabilirlik
- Hiperparametre ayarlama örneği (Keras Tuner)
- Sonuç ve çıktı (submission)

## 1.GPU KONTROLÜNÜN YAPILMASI

In [ ]:
import tensorflow as tf
import sys
print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("GPU listesi:", tf.config.list_physical_devices('GPU'))

## 2. KÜTÜPHANELER VE KULLANILACAKVERİ SETİ

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print("Input klasörleri:")
print(os.listdir("/kaggle/input"))

## 3. Dataset'i ekleyip dosyaları okuma.

In [ ]:
BASE = "/kaggle/input/plant-pathology-2020-fgvc7"
print("Dosyalar:", os.listdir(BASE))

# Train ve Test csv dosyalarını yükleyelim
train_df = pd.read_csv(os.path.join(BASE, "train.csv"))
test_df = pd.read_csv(os.path.join(BASE, "test.csv"))

print("Train CSV boyutu:", train_df.shape)
print(train_df.head())

print("Test CSV boyutu:", test_df.shape)
print(test_df.head())

## 4. Veri Analizi ve Görselleştirme (EDA)
Bu adımda:
- Train CSV dosyasındaki etiketlerden "label" sütunu türetilecek.
- Sınıf dağılımı grafiği çizilecek.
- Örnek görseller ekranda gösterilecek.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os

# label ile sütün ekliyoruz
# 4 etiket sütununu belirle
label_cols = ['healthy','multiple_diseases','rust','scab']

# Hangi sütun 1 ise o sınıf olacak
train_df['label'] = train_df[label_cols].idxmax(axis=1)

# Sınıf dağılımı
plt.figure(figsize=(6,4))
sns.countplot(data=train_df, x='label')
plt.title("Sınıf Dağılımı")
plt.show()

# Örnek 8 görsel göster
IMG_DIR = os.path.join(BASE, "images")

def show_images(df, n=8):
    sample = df.sample(n).reset_index(drop=True)
    plt.figure(figsize=(14,6))
    for i, row in sample.iterrows():
        img_path = os.path.join(IMG_DIR, row['image_id'] + ".jpg")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(2, n//2, i+1)
        plt.imshow(img)
        plt.title(row['label'])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

show_images(train_df, n=8)


In [ ]:
# Etiket sütununu yeniden oluştur
label_cols = ['healthy','multiple_diseases','rust','scab']
train_df['label'] = train_df[label_cols].idxmax(axis=1)

print(train_df[['image_id','label']].head())


## 5. Train / Validation Split
Veriyi %85 eğitim ve %15 doğrulama (validation) olacak şekilde ayırıyoruz.
Stratify parametresi ile sınıf dağılımının her iki sette de benzer kalmasını sağlıyoruz.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np, random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

train_sub, val_sub = train_test_split(
    train_df,
    test_size=0.15,
    stratify=train_df['label'],
    random_state=SEED
)

print("Train set boyutu:", len(train_sub))
print("Validation set boyutu:", len(val_sub))


## 6. Data Augmentation ve Generator
Modelin overfitting yapmaması için veri artırma (rotation, shift, zoom, flip) uygulayacağız.
Ardından `flow_from_dataframe` ile train/validation generator hazırlayacağız.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
BATCH_SIZE = 32

# Train augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation sadece normalize (rescale)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Dosya adlarını dataframe içine ekleyelim
train_sub['filename'] = train_sub['image_id'] + ".jpg"
val_sub['filename']   = val_sub['image_id'] + ".jpg"

# Generator tanımları
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_sub,
    directory=os.path.join(BASE, "images"),
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=val_sub,
    directory=os.path.join(BASE, "images"),
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False
)


## 7. Baseline CNN Modeli
Bu model sıfırdan küçük bir CNN yapısıdır. 
Amaç yüksek başarı değil, temel performansı görmek. 
Sonraki adımda Transfer Learning ile güçlü bir model kullanacağız.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

def build_baseline(input_shape=(224,224,3), n_classes=4):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inp)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    out = layers.Dense(n_classes, activation='softmax')(x)
    model = models.Model(inputs=inp, outputs=out)
    return model

# Modeli oluştur
baseline = build_baseline()

# Derleme
baseline.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model özetini yazdır
baseline.summary()

# Eğitim
EPOCHS = 6
history_base = baseline.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS
)


## 8. Eğitim Sonuçlarının Görselleştirilmesi
Baseline CNN modelinin eğitim sürecinde train/validation accuracy ve loss değerleri kaydedildi. 
Bu adımda bu değerleri grafik olarak çiziyoruz. 
Amaç, modelin öğrenme eğrisini görmek ve overfitting/underfitting eğilimlerini incelemek.

In [ ]:
import matplotlib.pyplot as plt

def plot_training(history):
    plt.figure(figsize=(12,4))

    # Accuracy grafiği
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title("Accuracy Eğrisi")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss grafiği
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title("Loss Eğrisi")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

plot_training(history_base)


## 9. Transfer Learning (ResNet50)
Bu adımda, ImageNet üzerinde eğitilmiş ResNet50 modelini kullanarak transfer learning uygulayacağız. 
Alt katmanlar dondurulacak, üst katmana kendi sınıflandırma katmanlarımız eklenecek.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Base model: ResNet50 (ImageNet ağırlıklarıyla, include_top=False)
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Katmanları dondur (önceden öğrendiklerini koru)
for layer in base_model.layers:
    layer.trainable = False

# Üst katmanları ekle
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.4)(x)
output = layers.Dense(4, activation='softmax')(x)

model_resnet = models.Model(inputs=base_model.input, outputs=output)

# Derleme
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Özet
model_resnet.summary()

# Eğitim
EPOCHS = 8
history_resnet = model_resnet.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS
)


## 10. Fine-Tuning ile ResNet50
ResNet50’nin alt katmanları dondurularak ilk eğitim yapıldı. 
Şimdi üst katmanların yanı sıra alt katmanların bir kısmını da eğitilebilir hale getiriyoruz (fine-tuning).

In [ ]:
# ResNet50 katmanlarının son 50 tanesini eğitilebilir hale getir
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Daha küçük learning rate
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Tekrar eğitim
EPOCHS_FINE = 10
history_fine = model_resnet.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS_FINE
)


## 11. Eğitim Sonuçlarının Görselleştirilmesi
Bu adımda ResNet50 (Fine-Tuned) modelinin train/validation accuracy ve loss eğrileri
gösterilmektedir. Grafikler üzerinden modelin öğrenme eğilimi ve olası overfitting durumu analiz edilmektedir.


In [ ]:
def plot_training(history, title="Model"):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12,4))

    # Accuracy grafiği
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f"{title} - Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss grafiği
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f"{title} - Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

plot_training(history_fine, title="ResNet50 (Fine-Tuned)")


In [ ]:
# Test CSV'yi yeniden yükle
import os
import pandas as pd

BASE = "/kaggle/input/plant-pathology-2020-fgvc7"

test_df = pd.read_csv(os.path.join(BASE, "test.csv"))
print(test_df.shape)
print(test_df.head())


## 12. Test Verisi Tahminleri ve Submission Dosyası
Bu adımda fine-tuned ResNet50 modeli ile test verisi üzerinde tahmin yapılacak.
Sonuçlar Kaggle formatında (`submission.csv`) kaydedilecek.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.4)(x)
output = layers.Dense(4, activation='softmax')(x)

model_resnet = models.Model(inputs=base_model.input, outputs=output)

model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os

BASE = "/kaggle/input/plant-pathology-2020-fgvc7"

# Tekrar tanımla
IMG_SIZE = 224
BATCH_SIZE = 32

# Test CSV yükle
test_df = pd.read_csv(os.path.join(BASE, "test.csv"))
test_df['filename'] = test_df['image_id'] + ".jpg"

# Test generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=os.path.join(BASE, "images"),
    x_col="filename",
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode=None,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Tahmin al
preds = model_resnet.predict(test_generator, verbose=1)

# Submission dataframe
submission = pd.DataFrame(
    preds,
    columns=['healthy','multiple_diseases','rust','scab']
)
submission.insert(0, 'image_id', test_df['image_id'])

# CSV olarak kaydet
submission.to_csv("submission.csv", index=False)
print("submission.csv oluşturuldu!")
submission.head()


## 13. EarlyStopping ve ModelCheckpoint ile Model Geliştirme

Modelin aşırı öğrenmesini (overfitting) engellemek için **EarlyStopping** kullandık.  
Bu yöntem, belirli sayıda epoch boyunca validation accuracy gelişmezse eğitimi durdurur 
ve en iyi ağırlıkları geri yükler.

Ayrıca **ModelCheckpoint** ile en iyi validation accuracy elde edilen epoch'ta model ağırlıkları kaydedildi.  
Böylece gereksiz uzun eğitimlerden kaçındık ve en iyi performansa sahip modeli saklamış olduk.

Bu aşamada learning rate düşük ayarlandı (`1e-5`) ve model toplam 20 epoch için eğitildi.  
Sonuçta modelin validation setindeki başarısı iyileşti, daha stabil bir öğrenme eğrisi elde edildi.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Modeli compile et (küçük lr ile)
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS_BONUS = 20
history_bonus = model_resnet.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS_BONUS,
    callbacks=[early_stop, checkpoint]
)

### EarlyStopping Sonuçları

Eğitim sırasında validation accuracy en yüksek değere **5. epoch**’ta ulaşmıştır (%38.32).  
EarlyStopping mekanizması sayesinde eğitim 8. epoch’ta durdurulmuş ve 
en iyi performansın elde edildiği ağırlıklar geri yüklenmiştir.

Bu yaklaşım, modelin **overfitting yapmasını önlemiş** ve gereksiz eğitim süresini kısaltmıştır.


## 14. Class Weights ile Veri Dengelenmesi

Veri setinde sınıflar arasında ciddi dengesizlik vardır:

- Rust: ~600 örnek
- Scab: ~500 örnek
- Healthy: ~500 örnek
- Multiple Diseases: ~100 örnek

Bu dengesizlik, modelin özellikle *multiple diseases* sınıfında düşük başarı göstermesine yol açmaktadır.  
Bunu dengelemek için `class_weight` parametresi kullanılmıştır.  

Sklearn'in `compute_class_weight` fonksiyonu ile her sınıfa ait ağırlıklar hesaplanmış, 
az örneğe sahip sınıflar daha yüksek ağırlıkla eğitilmiştir.  
Bu sayede model her sınıfa daha adil yaklaşır.


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Label değerlerini sayısal index haline getirelim
labels = train_generator.class_indices
print("Class Indices:", labels)

# Class weight hesapla
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Sözlük haline getir
class_weights_dict = {i: w for i, w in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# Modeli tekrar compile et
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Eğitim
EPOCHS_CW = 15
history_cw = model_resnet.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS_CW,
    class_weight=class_weights_dict,
    callbacks=[early_stop, checkpoint]
)


### Class Weights Sonuçları

Veri setindeki sınıf dengesizliği nedeniyle model bazı sınıflarda düşük başarı göstermekteydi.  
Bu problemi çözmek için `class_weight` kullanılmış ve az sayıda örneğe sahip sınıflara daha fazla önem verilmiştir.  

Sonuç olarak validation accuracy **%38.3’ten %41.6’ya** yükselmiştir.  
Bu, modelin özellikle **multiple_diseases** sınıfındaki performansını iyileştirdiğini göstermektedir.

EarlyStopping sayesinde eğitim 10. epoch’ta durdurulmuş ve en iyi epoch olan 7. epoch’un ağırlıkları geri yüklenmiştir.


## 15. Confusion Matrix ve Classification Report

Modelin performansını yalnızca accuracy ile değerlendirmek yeterli değildir.  
Bu nedenle her sınıf için precision, recall ve F1-score değerleri hesaplanmıştır.  
Ayrıca Confusion Matrix yardımıyla modelin hangi sınıflarda hata yaptığını daha net görebiliriz.

Bu analiz, modelin hangi bitki hastalıklarını doğru tanıyabildiğini, 
hangilerinde karışıklık yaşadığını ortaya koyar.


In [ ]:
from tensorflow.keras.models import load_model

# Kaydedilen en iyi modeli yükle
model_resnet = load_model("best_model.h5")

# Model yüklendiğinde tekrar test et
val_preds = model_resnet.predict(valid_generator)
val_preds_classes = np.argmax(val_preds, axis=1)
true_classes = valid_generator.classes
class_labels = list(valid_generator.class_indices.keys())

# Classification report
print("📊 Classification Report:")
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

print(classification_report(true_classes, val_preds_classes, target_names=class_labels))

# Confusion matrix
cm = confusion_matrix(true_classes, val_preds_classes)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix - Validation Set")
plt.show()


## 16. Model Performans Değerlendirmesi

Confusion Matrix ve Classification Report incelendiğinde modelin en iyi performansı **scab** sınıfında gösterdiği, 
ancak **multiple_diseases** sınıfında neredeyse hiç başarı elde edemediği görülmüştür.  

Bu durum, veri dengesizliğinin ve sınıflar arası görsel benzerliğin etkisini göstermektedir.  
Genel doğruluk oranı %39 civarındadır.

İyileştirme için şu adımlar önerilebilir:
- Veriyi arttırmak veya augmentasyon çeşitliliğini yükseltmek,  
- Az örnekli sınıflar için oversampling teknikleri uygulamak,  
- Alternatif derin öğrenme modelleri (DenseNet, EfficientNet) denemek,  
- Learning rate scheduler veya farklı optimizer stratejileriyle ince ayar yapmak.


### Confusion Matrix Görsel Analizi

Aşağıdaki Confusion Matrix grafiğinde modelin validation set üzerindeki tahmin performansı görselleştirilmiştir.  
Koyu mavi kareler doğru sınıflandırmaları, açık mavi kareler ise hatalı tahminleri temsil eder.

Model, özellikle **rust** ve **scab** sınıflarında doğru tahmin oranı daha yüksekken,  
**healthy** ve **multiple diseases** sınıflarında ciddi karışıklıklar gözlenmiştir.  
Bu durum sınıflar arası görsel benzerlik ve veri dengesizliği ile açıklanabilir.


## 17. Test Set Üzerinde Tahmin ve Submission Dosyası

Eğitilmiş ve en iyi ağırlıklara sahip model (`best_model.h5`) kullanılarak test verisi üzerinde tahminleme yapılmıştır.  
Bu tahmin sonuçları, her test görseli için dört sınıfa ait olasılık değerlerini içermektedir:

- healthy  
- multiple_diseases  
- rust  
- scab  

Oluşturulan sonuçlar `submission.csv` dosyasına kaydedilmiştir.  
Bu dosya Kaggle yarışmalarında değerlendirilebilecek biçimdedir.

In [ ]:
import os
BASE = "/kaggle/input/plant-pathology-2020-fgvc7"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os

# Test verisini yeniden hazırlayalım
test_df = pd.read_csv(os.path.join(BASE, "test.csv"))
test_df['filename'] = test_df['image_id'] + ".jpg"

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=os.path.join(BASE, "images"),
    x_col="filename",
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode=None,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Kaydedilen en iyi modeli yükle
from tensorflow.keras.models import load_model
model_resnet = load_model("best_model.h5")

# Tahmin al
preds = model_resnet.predict(test_generator, verbose=1)

# Submission dosyası oluştur
submission = pd.DataFrame(preds, columns=['healthy', 'multiple_diseases', 'rust', 'scab'])
submission.insert(0, 'image_id', test_df['image_id'])
submission.to_csv("submission.csv", index=False)

print("✅ Submission dosyası oluşturuldu!")
submission.head()


## 18. Sonuç ve Değerlendirme

Bu proje kapsamında bitki yaprak hastalıklarının sınıflandırılması için **ResNet50 tabanlı derin öğrenme modeli** geliştirilmiştir.  
Model, `Plant Pathology 2020 (FGVC7)` veri seti üzerinde eğitilmiş ve dört sınıf için tahmin yapmıştır:

- healthy  
- multiple_diseases  
- rust  
- scab  

Eğitim sürecinde aşağıdaki iyileştirmeler uygulanmıştır:

- **Transfer Learning:** ResNet50 önceden eğitilmiş ağırlıklarla başlatıldı.  
- **EarlyStopping:** Overfitting önlendi, gereksiz uzun eğitimler durduruldu.  
- **ModelCheckpoint:** En iyi epoch’a ait ağırlıklar kaydedildi.  
- **Class Weights:** Dengesiz veri dağılımı dengelendi.  
- **Confusion Matrix & Classification Report:** Modelin hangi sınıflarda güçlü/zayıf olduğu analiz edildi.

Sonuç olarak validation doğruluğu yaklaşık **%41.6** seviyesine ulaşmıştır.  
Model, `submission.csv` dosyasıyla Kaggle üzerinde değerlendirmeye hazır hale getirilmiştir.  

Proje boyunca veri analizi, model oluşturma, eğitim, değerlendirme ve çıktı üretim aşamaları tamamlanmıştır.
